## About Project

        1. Project Planning
        2. Project Estimation
        3. Resources Allocation


In [3]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

# Load environment variables
from dotenv import load_dotenv

load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

In [5]:
# Define file paths for YAML configurations
files = {"agents": "config/agents.yaml", "tasks": "config/tasks.yaml"}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

## Pydantic object

    -- so that all the output have the same structure


In [6]:
from typing import List
from pydantic import BaseModel, Field


class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(
        ..., description="Estimated time to complete the task in hours"
    )
    required_resources: List[str] = Field(
        ..., description="List of resources required to complete the task"
    )


class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(
        ..., description="List of task IDs associated with this milestone"
    )


class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(
        ..., description="List of tasks with their estimates"
    )
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Creating Crew AI


In [7]:
# Creating Agents
project_planning_agent = Agent(config=agents_config["project_planning_agent"])

estimation_agent = Agent(config=agents_config["estimation_agent"])

resource_allocation_agent = Agent(config=agents_config["resource_allocation_agent"])

# Creating Tasks
task_breakdown = Task(
    config=tasks_config["task_breakdown"], agent=project_planning_agent
)

time_resource_estimation = Task(
    config=tasks_config["time_resource_estimation"], agent=estimation_agent
)

resource_allocation = Task(
    config=tasks_config["resource_allocation"],
    agent=resource_allocation_agent,
    output_pydantic=ProjectPlan,  # This is the structured output we want
)

# Creating Crew
crew = Crew(
    agents=[project_planning_agent, estimation_agent, resource_allocation_agent],
    tasks=[task_breakdown, time_resource_estimation, resource_allocation],
    verbose=True,
)

## Crews Inputs


In [8]:
from IPython.display import display, Markdown

project = "Website"
industry = "Technology"
project_objectives = "Create a website for a small business"
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## kicking of the crew


In [9]:
# The given Python dictionary
inputs = {
    "project_type": project,
    "project_objectives": project_objectives,
    "industry": industry,
    "team_members": team_members,
    "project_requirements": project_requirements,
}

# Run the crew
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: The Ultimate Project Planner

 [INFO]: == Starting Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to sh

## How much will it cost??

        -- gpt 4o mini is like 15 cents per million tokens


In [20]:
crew.usage_metrics

{'total_tokens': 15966,
 'prompt_tokens': 8361,
 'completion_tokens': 7605,
 'successful_requests': 9}

In [18]:
import pandas as pd

costs = (
    0.150
    * (crew.usage_metrics["prompt_tokens"] + crew.usage_metrics["completion_tokens"])
    / 1_000_000
)
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics])
df_usage_metrics

Total costs: $0.0024


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,15966,8361,7605,9


## Output


In [26]:
result.dict()

{'tasks': [{'task_name': 'Lead requirements gathering, documentation, and initial planning',
   'estimated_time_hours': 40.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Participate in stakeholder meetings',
   'estimated_time_hours': 10.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Create designs, attend feedback sessions',
   'estimated_time_hours': 100.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Develop high-fidelity prototypes, conduct user testing',
   'estimated_time_hours': 120.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Develop navigation system and core page functionalities',
   'estimated_time_hours': 160.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Implement blog and testimonials features using CMS tools',
   'estimated_time_hours': 80.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Integrate SEO and social media functionalities',
   'estimated_time_hours': 40.0,
   'required_resources': 

In [27]:
tasks = result.dict()["tasks"]
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption(
    "Task Details"
).set_table_styles([{"selector": "th, td", "props": [("font-size", "120%")]}])

,task_name,estimated_time_hours,required_resources
0,"Lead requirements gathering, documentation, and initial planning",40.000000,['John Doe']
1,Participate in stakeholder meetings,10.000000,['Jane Doe']
2,"Create designs, attend feedback sessions",100.000000,['Bob Smith']
3,"Develop high-fidelity prototypes, conduct user testing",120.000000,['Bob Smith']
4,Develop navigation system and core page functionalities,160.000000,['Jane Doe']
5,Implement blog and testimonials features using CMS tools,80.000000,['Jane Doe']
6,Integrate SEO and social media functionalities,40.000000,['Jane Doe']
7,"Execute test cases, log bugs, and verify fixes",160.000000,"['Alice Johnson', 'Tom Brown']"
8,Oversee final preparations and manage go-live process,20.000000,['John Doe']
9,Assist in final deployment and troubleshooting,20.000000,['Jane Doe']


In [28]:
milestones = result.dict()["milestones"]
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption(
    "Task Details"
).set_table_styles([{"selector": "th, td", "props": [("font-size", "120%")]}])

,milestone_name,tasks
0,Project Initialization and Requirements Gathering,"['Lead requirements gathering, documentation, and initial planning', 'Participate in stakeholder meetings']"
1,Design Phase,"['Create designs, attend feedback sessions', 'Develop high-fidelity prototypes, conduct user testing']"
2,Development Phase,"['Develop navigation system and core page functionalities', 'Implement blog and testimonials features using CMS tools', 'Integrate SEO and social media functionalities']"
3,Quality Assurance,"['Execute test cases, log bugs, and verify fixes']"
4,Launch Preparation and Go-Live,"['Oversee final preparations and manage go-live process', 'Assist in final deployment and troubleshooting']"
5,Post-Launch Support and Maintenance,['Assist in final deployment and troubleshooting']
6,SEO Optimization and Social Media Integration,['Integrate SEO and social media functionalities']
